In [1]:
from svmutil import *

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [3]:
# df_data = pd.read_csv('X_train.csv', header=None)
# df_label = pd.read_csv('T_train.csv', header=None)
# labels = df_label.values
# df_data['label'] = labels
# df_data.to_csv('train.csv', index=False)

# # Execute CMD >> python csv2libsvm.py train.csv train.data 784 True

# df_data = pd.read_csv('X_test.csv', header=None)
# df_label = pd.read_csv('T_test.csv', header=None)
# labels = df_label.values
# df_data['label'] = labels
# df_data.to_csv('test.csv', index=False)

# # Execute CMD >> python csv2libsvm.py test.csv test.data 784 True

In [4]:
y_train, X_train = svm_read_problem(r'train.data')
y_test, X_test = svm_read_problem(r'test.data')

In [13]:
# Use different kernel functions  (linear, polynomial, and RBF kernels) and have comparison between their performance. 
print('Linear SVM:')
linear_model =  svm_train(y_train, X_train, '-t 0' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, linear_model)

print('\nPolynoimal SVM:')
poly_model =  svm_train(y_train, X_train, '-t 1' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, poly_model)

print('\nRBF SVM:')
RBF_model =  svm_train(y_train, X_train, '-t 2' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, RBF_model)

Linear SVM:
Accuracy = 95.08% (2377/2500) (classification)

Polynoimal SVM(degree=1):
Accuracy = 94.8% (2370/2500) (classification)

Polynoimal SVM(degree=2):
Accuracy = 88.24% (2206/2500) (classification)

Polynoimal SVM(degree=3):
Accuracy = 34.68% (867/2500) (classification)

RBF SVM:
Accuracy = 95.32% (2383/2500) (classification)


In [9]:
# use C-SVCdo the grid search
RBF_model =  svm_train(y_train, X_train, '-t 2 -c 32.0  -g 0.0078125' )
print('RBF SVM:')
p_label, p_acc, p_val = svm_predict(y_test,  X_test, RBF_model)

RBF SVM:
Accuracy = 98.04% (2451/2500) (classification)


In [7]:
help(svm_train)

Help on function svm_train in module svmutil:

svm_train(arg1, arg2=None, arg3=None)
    svm_train(y, x [, options]) -> model | ACC | MSE
    
    y: a list/tuple/ndarray of l true labels (type must be int/double).
    
    x: 1. a list/tuple of l training instances. Feature vector of
          each training instance is a list/tuple or dictionary.
    
       2. an l * n numpy ndarray or scipy spmatrix (n: number of features).
    
    svm_train(prob [, options]) -> model | ACC | MSE
    svm_train(prob, param) -> model | ACC| MSE
    
    Train an SVM model from data (y, x) or an svm_problem prob using
    'options' or an svm_parameter param.
    If '-v' is specified in 'options' (i.e., cross validation)
    either accuracy (ACC) or mean-squared error (MSE) is returned.
    options:
        -s svm_type : set type of SVM (default 0)
            0 -- C-SVC              (multi-class classification)
            1 -- nu-SVC             (multi-class classification)
            2 -- one-class

In [15]:
from sklearn.metrics.pairwise import rbf_kernel

def create_kernel(x1, x2, lam_1=1, lam_2=1): # train/train or train/test
    # create dense data for x1 (train)
    max_key=np.max([np.max(v.keys()) for v in x1])
    arr=np.zeros( (len(x1), len(max_key) ))
    for row, vec in enumerate(x1):
        for k, v in vec.items():
            arr[row][k-1]=v
    x1 = arr
    
    #create dense data for x2
    max_key=np.max([np.max(v.keys()) for v in x2])
    arr=np.zeros( (len(x2), len(max_key) ))
    for row, vec in enumerate(x2):
        for k, v in vec.items():
            arr[row][k-1]=v
    x2 = arr
    
    #create a linear kernel matrix 
    k_linear = np.zeros( (len(x2), len(x1)) )
    k_linear = np.dot(x2, x1.T)
    
    #create a RBF kernel matrix 
    k_RBF = np.zeros( (len(x2), len(x1)) )
    k_RBF = rbf_kernel(x2, x1)
    
    #create kernel matrix 
    k = np.zeros( (len(x2), len(x1)+1) )
    k[:,1:] = 0*k_linear + lam_2*k_RBF
    k[:,:1] = np.arange(len(x2))[:,np.newaxis]+1
    
    kernel = [list(row) for row in k]
    return kernel


x_train = X_train[:]
lam_1_range = [0]
lam_2_range = [1.0]
for lam_1 in lam_1_range:
    for lam_2 in lam_2_range:
        if lam_1 == lam_2 and lam_1!= 1: continue
        print("lam_1:%.1f, lam_2:%.1f"%(lam_1, lam_2), end = ' | ')
        kernel = create_kernel(x_train, x_train, lam_1, lam_2)
        model = svm_train(y_train, kernel, '-t 4 -c 3.0  -g 0.01')
        x_test = X_test[:]
        kernel = create_kernel(x_train, x_test, lam_1, lam_2)
        p_label, p_acc, p_val = svm_predict(y_test, kernel, model)

lam_1:0.0, lam_2:1.0 | Accuracy = 96.04% (2401/2500) (classification)


In [5]:
def grid_search(y_train, X_train, args='-t 2 -v 5 -q', para_range= {'c':[1,2,3], 'g':[None], 'd':[None], 'r':[None]} ):
    best_acc = 0.0
    best_para = {'c':0, 'g':0, 'd':0, 'r':0}
    
    pbar = tqdm(total = len(para_range['c']) * len(para_range['c']) * len(para_range['d']) * len(para_range['r']) )
    for c in para_range['c']:
        for g in para_range['g']:
            for d in para_range['d']:
                for r in para_range['r']:
                    arg_c = '' if c is None else ' -c %s '%str(c)
                    arg_g = '' if g is None else ' -g %s '%str(g)
                    arg_d = '' if d is None else ' -d %s '%str(d)
                    arg_r = '' if r is None else ' -r %s '%str(r)
                    args_ = args + arg_c + arg_g + arg_d + arg_r  
                    print(args_, end=' | ')
                    acc = svm_train(y_train, X_train, args_)
                    if acc > best_acc:
                        best_acc = acc
                        best_para = {'c':c, 'g':g, 'd':d, 'r':r}
                    pbar.update()
    pbar.close()
    print("Best Parameters ", best_para , 'Accuracy :', best_acc)

In [15]:
# RBF
grid_search(y_train, X_train, args='-t 2 -v 5', para_range= {'c':[1,2,3,4,5], 'g':[1e-1, 1e-2, 1e-3, 1e-4], 'd':[None], 'r':[None]})

-t 2 -v 5 -c 1  -g 0.1  | Cross Validation Accuracy = 91.88%
-t 2 -v 5 -c 1  -g 0.01  | Cross Validation Accuracy = 97.86%
-t 2 -v 5 -c 1  -g 0.001  | Cross Validation Accuracy = 96.2%
-t 2 -v 5 -c 1  -g 0.0001  | Cross Validation Accuracy = 92.66%
-t 2 -v 5 -c 2  -g 0.1  | Cross Validation Accuracy = 92.38%
-t 2 -v 5 -c 2  -g 0.01  | Cross Validation Accuracy = 98.08%
-t 2 -v 5 -c 2  -g 0.001  | Cross Validation Accuracy = 96.54%
-t 2 -v 5 -c 2  -g 0.0001  | Cross Validation Accuracy = 94.26%
-t 2 -v 5 -c 3  -g 0.1  | Cross Validation Accuracy = 92.34%
-t 2 -v 5 -c 3  -g 0.01  | Cross Validation Accuracy = 98.2%
-t 2 -v 5 -c 3  -g 0.001  | Cross Validation Accuracy = 96.86%
-t 2 -v 5 -c 3  -g 0.0001  | Cross Validation Accuracy = 95.02%
-t 2 -v 5 -c 4  -g 0.1  | Cross Validation Accuracy = 92.48%
-t 2 -v 5 -c 4  -g 0.01  | Cross Validation Accuracy = 98%
-t 2 -v 5 -c 4  -g 0.001  | Cross Validation Accuracy = 97.02%
-t 2 -v 5 -c 4  -g 0.0001  | Cross Validation Accuracy = 95.4%
-t 2 -

-t 1 -v 5 -c 1  -g 0.01  -d 3  -r 0  | Cross Validation Accuracy = 96.44%
-t 1 -v 5 -c 1  -g 0.01  -d 3  -r 1  | Cross Validation Accuracy = 97.84%
-t 1 -v 5 -c 1  -g 0.01  -d 2  -r 0  | Cross Validation Accuracy = 97.56%
-t 1 -v 5 -c 1  -g 0.01  -d 2  -r 1  | Cross Validation Accuracy = 97.48%
-t 1 -v 5 -c 1  -g 0.01  -d 1  -r 0  | Cross Validation Accuracy = 96.9%
-t 1 -v 5 -c 1  -g 0.01  -d 1  -r 1  | Cross Validation Accuracy = 96.88%
-t 1 -v 5 -c 1  -g 0.1  -d 3  -r 0  | Cross Validation Accuracy = 97.68%
-t 1 -v 5 -c 1  -g 0.1  -d 3  -r 1  | Cross Validation Accuracy = 98%
-t 1 -v 5 -c 1  -g 0.1  -d 2  -r 0  | Cross Validation Accuracy = 98.04%
-t 1 -v 5 -c 1  -g 0.1  -d 2  -r 1  | Cross Validation Accuracy = 97.84%
-t 1 -v 5 -c 1  -g 0.1  -d 1  -r 0  | Cross Validation Accuracy = 96.84%
-t 1 -v 5 -c 1  -g 0.1  -d 1  -r 1  | Cross Validation Accuracy = 96.78%
-t 1 -v 5 -c 2  -g 0.01  -d 3  -r 0  | Cross Validation Accuracy = 97.22%
-t 1 -v 5 -c 2  -g 0.01  -d 3  -r 1  | Cross Val


Params: -t 0 -v 5 -q -c 1
Cross Validation Accuracy = 96.16%

Params: -t 0 -v 5 -q -c 2
Cross Validation Accuracy = 96.64%

Params: -t 0 -v 5 -q -c 3
Cross Validation Accuracy = 96.24%

Params: -t 0 -v 5 -q -c 4
Cross Validation Accuracy = 96.54%

Params: -t 0 -v 5 -q -c 5
Cross Validation Accuracy = 95.96%

Best Parameters : [C=2.000000, None=0.000000] Accuracy : 96.64


In [13]:
# use para found in grid search 
# print('Linear SVM:')
# linear_model =  svm_train(y_train, X_train, '-t 0 -c 2.0' )
# p_label, p_acc, p_val = svm_predict(y_test,  X_test, linear_model)

# print('\nPolynoimal SVM:')
# poly_model =  svm_train(y_train, X_train, '-t 1 -c 2  -g 0.1  -d 2  -r 0' )
# p_label, p_acc, p_val = svm_predict(y_test,  X_test, poly_model)

print('\nRBF SVM:')
RBF_model =  svm_train(y_train, X_train, '-t 2 -c 3.0  -g 0.01' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, RBF_model)


RBF SVM:
Accuracy = 98.08% (2452/2500) (classification)


In [6]:
from sklearn.metrics.pairwise import rbf_kernel

def create_kernel(x1, x2, lam_1=1, lam_2=1): # train/train or train/test
    # create dense data for x1 (train)
    max_key=np.max([np.max(v.keys()) for v in x1])
    arr=np.zeros( (len(x1), len(max_key) ))
    for row, vec in enumerate(x1):
        for k, v in vec.items():
            arr[row][k-1]=v
    x1 = np.copy(arr)
    
    #create dense data for x2
    max_key=np.max([np.max(v.keys()) for v in x2])
    arr=np.zeros( (len(x2), len(max_key) ))
    for row, vec in enumerate(x2):
        for k, v in vec.items():
            arr[row][k-1]=v
    x2 = np.copy(arr)
    
    #create a linear kernel matrix 
    k_linear = np.zeros( (len(x2), len(x1)) )
    k_linear = np.dot(x2, x1.T)
    
    #create a RBF kernel matrix 
    k_RBF = np.zeros( (len(x2), len(x1)) )
    k_RBF = rbf_kernel(x2, x1, gamma=0.01)
    
    #create kernel matrix 
    k = np.zeros( (len(x2), len(x1)+1) )
    k[:,1:] = lam_1*k_linear + lam_2*k_RBF
    k[:,:1] = np.arange(len(x2))[:,np.newaxis]+1
    
    kernel = [list(row) for row in k]
    return kernel



lam_1_range = [0.2, 0.4, 0.6, 0.8, 1.0]
lam_2_range = [0.2, 0.4, 0.6, 0.8, 1.0]
for lam_1 in lam_1_range:
    for lam_2 in lam_2_range:
        if lam_1 == lam_2 and lam_1!= 1: continue
        print("lam_1:%.1f, lam_2:%.1f"%(lam_1, lam_2), end = ' | ')
        kernel = create_kernel(X_train, X_train, lam_1, lam_2)
        model = svm_train(y_train, kernel, '-t 4 -c 3 -g 0.01')
        kernel = create_kernel(X_train, X_test, lam_1, lam_2)
        p_label, p_acc, p_val = svm_predict(y_test, kernel, model)

lam_1:0.2, lam_2:0.4 | Accuracy = 95.72% (2393/2500) (classification)
lam_1:0.2, lam_2:0.6 | Accuracy = 95.76% (2394/2500) (classification)
lam_1:0.2, lam_2:0.8 | Accuracy = 95.92% (2398/2500) (classification)
lam_1:0.2, lam_2:1.0 | Accuracy = 96% (2400/2500) (classification)
lam_1:0.4, lam_2:0.2 | Accuracy = 95.12% (2378/2500) (classification)
lam_1:0.4, lam_2:0.6 | Accuracy = 95.44% (2386/2500) (classification)
lam_1:0.4, lam_2:0.8 | Accuracy = 95.48% (2387/2500) (classification)
lam_1:0.4, lam_2:1.0 | Accuracy = 95.48% (2387/2500) (classification)
lam_1:0.6, lam_2:0.2 | Accuracy = 95.04% (2376/2500) (classification)
lam_1:0.6, lam_2:0.4 | Accuracy = 95.16% (2379/2500) (classification)
lam_1:0.6, lam_2:0.8 | Accuracy = 95.4% (2385/2500) (classification)
lam_1:0.6, lam_2:1.0 | Accuracy = 95.44% (2386/2500) (classification)
lam_1:0.8, lam_2:0.2 | Accuracy = 95% (2375/2500) (classification)
lam_1:0.8, lam_2:0.4 | Accuracy = 95.12% (2378/2500) (classification)
lam_1:0.8, lam_2:0.6 | Accu

In [37]:
print('\nRBF SVM:')

RBF_model =  svm_train(y_train, X_train, '-t 2 -c 3.0  -g 0.01' )
p_label, p_acc, p_val = svm_predict(y_test,  X_test, RBF_model)
# print("Best Parameters : {'c': 2, 'g': 0, 'd': 0, 'r': 0} Accuracy : 96.64")
# print("Best Parameters  {'c': 2, 'g': 0.1, 'd': 2, 'r': 0} Accuracy : 98.14")
# print("Best Parameters  {'c': 3, 'g': 0.01, 'd': 0, 'r': 0} Accuracy : 98.57")


RBF SVM:
Accuracy = 98.08% (2452/2500) (classification)


In [33]:
# kernel_train = create_kernel(X_train, X_train, lam_1, lam_2)

# kernel_test = create_kernel(X_train, X_test, lam_1, lam_2)

model = svm_train(y_train, kernel_train, '-t 4 -c 3.0 -g 0.02')
p_label, p_acc, p_val = svm_predict(y_test, kernel_test, model)

Accuracy = 96.04% (2401/2500) (classification)
